In [1]:
!pip install azure-storage-blob

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 394.5/394.5 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.4/193.4 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 2.2 MB/s eta 0:00:00


In [2]:
# import Librairies
import pandas as pd
import numpy as np
import json
import requests
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient
from io import StringIO

In [3]:
URL = "https://data.cityofnewyork.us/resource/erm2-nwe9.csv"

In [4]:
limit = 100000
URL2 = f'{URL}?$limit={limit}'

In [5]:
df_raw = pd.read_csv(URL2)
df_raw.head()

<ipython-input-5-53c00a5b18c7>:1: DtypeWarning: Columns (18) have mixed types. Specify dtype option on import or set low_memory=False.
  df_raw = pd.read_csv(URL2)


,unique_key,created_date,closed_date,agency,agency_name,complaint_type,descriptor,location_type,incident_zip,incident_address,...,vehicle_type,taxi_company_borough,taxi_pick_up_location,bridge_highway_name,bridge_highway_direction,road_ramp,bridge_highway_segment,latitude,longitude,location
0,60903704,2024-04-19T01:58:13.000,NaN,DOT,Department of Transportation,Street Condition,Pothole,NaN,11211.0,METROPOLITAN AVENUE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,60914765,2024-04-19T01:51:21.000,NaN,NYPD,New York City Police Department,Noise - Commercial,Loud Music/Party,Club/Bar/Restaurant,10013.0,40 MULBERRY STREET,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.714837,-73.999614,"\n, \n(40.71483730104454, -73.99961402373336)"
2,60914650,2024-04-19T01:51:19.000,NaN,NYPD,New York City Police Department,Blocked Driveway,Partial Access,Street/Sidewalk,11221.0,831 MONROE STREET,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.688651,-73.922958,"\n, \n(40.68865115071951, -73.92295765130292)"
3,60906953,2024-04-19T01:50:56.000,NaN,NYPD,New York City Police Department,Illegal Parking,Blocked Hydrant,Street/Sidewalk,11210.0,2708 BEDFORD AVENUE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.636832,-73.953632,"\n, \n(40.63683244425841, -73.95363249985976)"
4,60908518,2024-04-19T01:50:49.000,NaN,NYPD,New York City Police Department,Noise - Commercial,Loud Music/Party,Store/Commercial,10033.0,4325 BROADWAY,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.852371,-73.934628,"\n, \n(40.85237108974576, -73.93462789776359)"


In [6]:
df_raw.columns

Index(['unique_key', 'created_date', 'closed_date', 'agency', 'agency_name',
       'complaint_type', 'descriptor', 'location_type', 'incident_zip',
       'incident_address', 'street_name', 'cross_street_1', 'cross_street_2',
       'intersection_street_1', 'intersection_street_2', 'address_type',
       'city', 'landmark', 'facility_type', 'status', 'due_date',
       'resolution_description', 'resolution_action_updated_date',
       'community_board', 'bbl', 'borough', 'x_coordinate_state_plane',
       'y_coordinate_state_plane', 'open_data_channel_type',
       'park_facility_name', 'park_borough', 'vehicle_type',
       'taxi_company_borough', 'taxi_pick_up_location', 'bridge_highway_name',
       'bridge_highway_direction', 'road_ramp', 'bridge_highway_segment',
       'latitude', 'longitude', 'location'],
      dtype='object')

In [7]:
df_raw.shape

(100000, 41)

In [8]:
df_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 41 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   unique_key                      100000 non-null  int64  
 1   created_date                    100000 non-null  object 
 2   closed_date                     72890 non-null   object 
 3   agency                          100000 non-null  object 
 4   agency_name                     100000 non-null  object 
 5   complaint_type                  100000 non-null  object 
 6   descriptor                      97083 non-null   object 
 7   location_type                   86431 non-null   object 
 8   incident_zip                    98897 non-null   float64
 9   incident_address                95935 non-null   object 
 10  street_name                     95932 non-null   object 
 11  cross_street_1                  72559 non-null   object 
 12  cross_street_2   

In [9]:
df_raw.dropna()
df_raw.shape

(100000, 41)

In [10]:
df_cleaned = df_raw.copy()

In [12]:
df_cleaned = df_raw.drop(columns = ['road_ramp','location_type','incident_address','cross_street_1','cross_street_2','intersection_street_1','intersection_street_2','address_type','landmark','facility_type','due_date','community_board','bbl','x_coordinate_state_plane','y_coordinate_state_plane','park_facility_name','park_borough','vehicle_type','taxi_company_borough','taxi_pick_up_location','bridge_highway_name','bridge_highway_direction','road_ramp','bridge_highway_segment'])

In [13]:
df_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 18 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   unique_key                      100000 non-null  int64  
 1   created_date                    100000 non-null  object 
 2   closed_date                     72890 non-null   object 
 3   agency                          100000 non-null  object 
 4   agency_name                     100000 non-null  object 
 5   complaint_type                  100000 non-null  object 
 6   descriptor                      97083 non-null   object 
 7   incident_zip                    98897 non-null   float64
 8   street_name                     95932 non-null   object 
 9   city                            95077 non-null   object 
 10  status                          100000 non-null  object 
 11  resolution_description          87627 non-null   object 
 12  resolution_action

In [14]:
from io import BytesIO, StringIO

# Azure Functions
def azure_upload_blob(connect_str, container_name, blob_name, data):
    blob_service_client = BlobServiceClient.from_connection_string(connect_str)
    blob_client = blob_service_client.get_blob_client(container=container_name, blob=blob_name)
    blob_client.upload_blob(data, overwrite=True)
    print(f"Uploaded to Azure Blob: {blob_name}")

def azure_download_blob(connect_str, container_name, blob_name):
    blob_service_client = BlobServiceClient.from_connection_string(connect_str)
    blob_client = blob_service_client.get_blob_client(container=container_name, blob=blob_name)
    download_stream = blob_client.download_blob()
    return download_stream.readall()

In [15]:
#specify the path to your JSON configuration file
config_file_path = 'config.json'

#Load the JSON configuration file
with open(config_file_path,'r') as config_file:
    config=json.load(config_file)

#Print the configuration
#Connection_STRING = config["connectionString"]

In [16]:
#Create Azure account, Azure storage
#Create container, copy access keys
CONNECTION_STRING_AZURE_STORAGE = config['connectionString']
CONTAINER_AZURE = 'homework1'

In [17]:
blob_name = "homework1_311_20240420.csv"

# Convert DataFrame to CSV
output = StringIO()
df_raw.to_csv(output, index=False)
data = output.getvalue()
output.close()

# Create the BlobServiceClient object
blob_service_client = BlobServiceClient.from_connection_string(CONNECTION_STRING_AZURE_STORAGE)

# Get a blob client using the container name and blob name
blob_client = blob_service_client.get_blob_client(container=CONTAINER_AZURE, blob=blob_name)

# Upload the CSV data
blob_client.upload_blob(data, overwrite=True)

print(f"Uploaded {blob_name} to Azure Blob Storage in container {CONTAINER_AZURE}.")

Uploaded homework1_311_20240420.csv to Azure Blob Storage in container homework1.
